<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Naas Lab

## Input

### Import libraries

In [ ]:
import pickle
import os

## Model

### Pickel functions

#### Pickle dump

In [ ]:
def pdump(
    output_dir,
    object_to_dump,
    file_to_dump_to,
):
    os.makedirs(output_dir, exist_ok=True)
    file_path = os.path.join(output_dir, f'{file_to_dump_to}.pickle')
    with open(file_path, 'wb') as file:
        pickle.dump(object_to_dump, file)

#### Pickle load

In [ ]:
def pload(
    output_dir,
    file_to_load_from
):
    file_path = os.path.join(output_dir, f'{file_to_load_from}.pickle')
    try:
        with open(file_path, 'rb') as file:
            return pickle.load(file)
    except:
        return None

## Output